# Week 10

* %SYSFUNC Function
* %QSYSFUNC Function

[Executing SAS Functions with the %SYSFUNC Macro Function](https://www.beoptimized.be/pdf/RLSPSYSF_001.pdf)



### %SYSFUNC function executes most of the data step functions. 

“Because %SYSFUNC is a macro function, you do not need to enclose character values in quotation marks as you do in DATA step functions” (SAS® Documentation).


In [1]:
* Ex4_Percent_sysfunc.sas;
options nodate nonumber nonotes nosource;
ods html close;

**** Here is a DATA step ;
data _null_;
  ds1 ='CLASS|CARS|HEART|DEMOGRAPHICS';
  count_words_v1 = countw(ds1, '|');
  put 'DATA step variable: ' DS1=;
  put 'DATA step variable: ' count_words_v1= ;
run;

SAS Connection established. Subprocess id is 11592



In [6]:
**** Creating and displaying macro variables in open code;
options nodate nonumber nosource;
ods html close;
%let m_ds1 = CLASS|CARS|HEART|DEMOGRAPHICS;
%let m_count_words_v1 = %sysfunc(countw(&m_ds1, %str(|)) );
%put Macro variable: &=m_ds1;
%put Macro variable: &=m_count_words_v1;
%symdel m_ds1 m_count_words_v1/nowarn;
%put _user_;

#### Things to notice in the SAS code in the following cell.

* The macro variable m_ds2 has a comma (,) in its value.
* The comma (,) is part of the macro variable value, not a delemter.

#### Question: How to mask the comma (,) - a special character (or token)?

#### Answer: Use the %quote macro quoting function to reference &m_ds2.
If you don't use %quote, SAS would treat the comma as a delimiter, which is meaningful in the countw function.

[Read Patterson and Remigio, 2017 about macro quoting functions](https://support.sas.com/resources/papers/proceedings/proceedings/forum2007/152-2007.pdf)



In [7]:
options nodate nonumber nosource;
ods html close;

%let m_ds3 = "CLASS","CARS","HEART","DEMOGRAPHICS";
%let m_count_words_v3 = %sysfunc(countw(%quote(&m_ds3)));
%put Macro variable: &=m_ds3;
%put Macro variable: &=m_count_words_v3;
%put _user_;
%symdel m_ds3 m_count_words_v3/nowarn;
%put _user_;

In [ ]:
options nonumber nocenter nodate nosource nosymbolgen;
ods html close;
%LET list = %str(sashelp.class sashelp.iris sashelp.retail);
/* Count # of values in the string */
%LET count=%sysfunc(countw(&list, %STR( ))); 
%put &=list;
%put &=count;
%macro doit;
  %do i = 1 %to &count;
    %put title&i  %left(%unquote(%SCAN(&list, &i, %STR( ))));
   %end;
   %put _user_;
%mend doit;
%doit

### Another example of %SYSFUNC 


In [4]:
title;
options nodate nonumber nosource nonotes;
ods html close;
footnote1 "%sysfunc(date(),worddate.) Class Report";
proc print data=sashelp.class (obs=3);
run;
%let string1 = %sysfunc(translate(&string1,P, N));


Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
2,Alice,F,13,56.5,84.0
3,Barbara,F,13,65.3,98.0


In [5]:
options nodate nonumber nosource;
ods html close;
data _null_;
    var1= 'STAT 4197';
    var2 = translate(var1, '6', '4');
    put 'DATA step variable:' var1= ; 
    put 'DATA step variable:' var2= ;  
run;
%let m_var1 = STAT 4197;
%let m_var2 = %sysfunc(translate(&m_var1, 6, 4));
%put macro variable: &=m_var1;
%put macro variable: &=m_var2;
%symdel var1 var2 /nowarn;



##### %SYSFUNC and PUTN only take macro variables, not data step variables.


In [85]:
options nodate nonumber nosource;
ods html close;
%let x1=%sysfunc(today(),date9.);
%let x2=%sysfunc(today(),monname3.);
%let x3=%sysfunc(today(),day2.);
%let x4=%sysfunc(today(),year4.);
%let x5 =%sysfunc(putn('27jan2016'd, date9.)); 
%let x6 =%sysfunc(putn(0, date9.)); 
%let x7 =%sysfunc(inputn(04jul2016, date9.)); 
%let x7 =%sysfunc(putn(%sysfunc(inputn(04jul2016, date9.)), mmddyy10.)); 
%let x9=%sysfunc(datetime(), datetime.);
%let x10=%sysfunc(date(),worddate.);
%let x11= %sysfunc(left(%qsysfunc(date(),worddate32.)));
%put &=x1 &=x2 &=x3 &=x4 &=x5 &=x6 ;
%put &=x7 &=x8 &=x9;
%put &=x10 &=x11;
%symdel x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11/nowarn;


[Author: KurtBremser Call INTNX FROM %SYSFUNC](https://communities.sas.com/t5/SAS-Programming/Call-INTNX-FROM-SYSFUNC-issue/td-p/379671)

In [20]:
options nonotes nodate nonumber nosource;
ods html close;
%put %sysfunc(intnx(month,%sysfunc(today()),1),monname8.);
%put %sysfunc(intnx(month,%sysfunc(today()),-1,s),year.);
%put %sysfunc(intnx(year,%sysfunc(today()),2),year.);

[The following code is obtained from author Haikuo at Communities.sas.com - 08/09/2013](https://communities.sas.com/t5/SAS-Programming/countw-and-double-quoted-string-list/td-p/106084)



In [ ]:
*SAS Documentation 9.4;
%macro checkds(dsn);
   %if %sysfunc(exist(&dsn)) %then
      %do;
         proc print data=&dsn;
         run;
      %end;
      %else
         %put The data set &dsn does not exist.;
%mend checkds;
%checkds(Sasuser.Houses)

##### Challenge
[How to replace commas in a macro variable and put spaces](https://communities.sas.com/t5/SAS-Programming/replace-commas-in-a-macro-variable-and-put-spaces/td-p/357639)
```
 "You will need to macro quote both commas and spaces to get them to be treated as text instead of delimiters." (Solution by Tom SUPER USER)
```

The %SYSFUNC enables the macro facility to execute SAS language functions or user-written funtions in open code (SAS Documentation).


In [24]:
options nocenter nonumber nodate nonotes nosource;
ods html close;
%let have = a,b,c;
%let want = %sysfunc(translate(%quote(&have),%str( ),%str(,)));
%put &=want;

In [32]:
options nocenter nonumber nodate nonotes nosource;
ods html close;
data _null_;
have = 'a,b,c';
want = translate(have, ' ', ',');
put (_ALL_) (=);

##### %QSYSFUNC function additionally masks special characters and mnemonic operators.
```
%SYSFUNC does not mask special characters or mnemonic operators in its result. %QSYSFUNC masks the following special characters and mnemonic operators in its result:
& % ' " ( ) + − * / < > = ¬ ^ ~ ; , #  blank
AND OR NOT EQ NE LE LT GE GT IN

```

In the example below,

* the %SYSFUNC executes the COMPBL function to remove multiple blanks in the character string by translating each occurrence of two or more consecutive blanks into a single blank.

* the %QSYSFUNC is used to mask the quotes.
If you do not the %QSYSFUNC masks, you will get this note.
```
NOTE 49-169: The meaning of an identifier after a quoted string might

             change in a future SAS release.  Inserting white space

             between a quoted string and the succeeding identifier is

             recommended.
```

In [12]:
options nocenter nonumber nodate nonotes nosource;
ods html close;
%let asthma_vars17 =    ASTHDX  ASSTIL31   ASATAK31  PERWT17F VARSTR VARPSU;
%let q_asthma_vars17 = %unquote(%str(%')%qsysfunc(tranwrd(%sysfunc(compbl(&asthma_vars17)),%str( ),' '))%str(%'));
%put &=asthma_vars17;
%put &=q_asthma_vars17;

When a macro variable is created with a macro quoting function like %STR or %QSYSFUNC, the function uses unprintable delta characters to mask the special characters in the macro variable's value.  

These delta characters can cause unexpected syntax errors when the macro variable is used in non-macro syntax.  Below, the macro variable has resolved and the syntax error occured during tokenization.

Solution: Use the %UNQUOTE function to removes the delta characters and prevent possible syntax errors caused by the characters.

In [19]:
options nocenter nonumber nodate nonotes nosource;
ods html close;
%let asthma_vars17 =    ASTHDX  ASSTIL31   ASATAK31  PERWT17F VARSTR VARPSU;
%let s_asthma_vars17 = %str(%')%qsysfunc(tranwrd(%sysfunc(compbl(&asthma_vars17)),%str( ),' '))%str(%');
%put _user_;

See Ex6_SYSFUNC.SAS for additional example-codes in GitHub folder  \SASCourse\Week10\SAS_Codes.